In [ ]:
#!pip install bitsandbytes

In [ ]:
#!pip install trl==0.21.0

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig

tokenizer = AutoTokenizer.from_pretrained("google/gemma-3-270m")
model = AutoModelForCausalLM.from_pretrained("google/gemma-3-270m")
generation_config = GenerationConfig.from_model_config(model.config)

generation_config.temperature = 0
generation_config.do_sample = True

In [ ]:
import transformers
#transformers.logging.set_verbosity_info()
input_ids = tokenizer.encode("Hello", return_tensors="pt")
generated_ids = model.generate(input_ids, generation_config=generation_config)
tokenizer.decode(generated_ids[0], skip_special_tokens=True)

In [ ]:
sql_prompt = "What is the total volume of timber sold by each salesperson, sorted by salesperson?"
table_ddl = "CREATE TABLE salesperson (salesperson_id INT, name TEXT, region TEXT); INSERT INTO salesperson (salesperson_id, name, region) VALUES (1, 'John Doe', 'North'), (2, 'Jane Smith', 'South'); CREATE TABLE timber_sales (sales_id INT, salesperson_id INT, volume REAL, sale_date DATE); INSERT INTO timber_sales (sales_id, salesperson_id, volume, sale_date) VALUES (1, 1, 120, '2021-01-01'), (2, 1, 150, '2021-02-01'), (3, 2, 180, '2021-01-01');"

template = """{% for m in messages -%}
{{ '<|'+m['role']+'|>\\n' + m['content'] + '\\n' }}
{%- endfor %}<|assistant|>"""


messages = [
    {"role": "system", "content": "You are a helpful assistant skilled in translating natural language into SQL Queries given the Table DDL. You will respond only with the correct MySQL Query."},
    {"role": "user", "content": f"Text: {sql_prompt}; \nDDL: {table_ddl}"},
]

tokenizer.chat_template=template
chat = tokenizer.apply_chat_template(messages, tokenize=True, return_tensors="pt")

# Supervised Fine Tuning

## Data Preparation

In [ ]:
from datasets import load_dataset

ds = load_dataset("gretelai/synthetic_text_to_sql")

In [ ]:
ds["train"][0].keys()

In [ ]:
user_prompt = """Given the <USER_QUERY> and the <SCHEMA>, generate the corresponding SQL command to retrieve the desired data, considering the query's syntax, semantics, and schema constraints.

<SCHEMA>
{context}
</SCHEMA>

<USER_QUERY>
{question}
</USER_QUERY>
"""

In [ ]:
def create_conversation(example):
    return {
        "messages": [
            {"role": "user", "content": user_prompt.format(question=example["sql_prompt"], context=example["sql_context"])},
            {"role": "assistant", "content": example["sql"]}
        ]
    }

In [ ]:
features = ds["train"].features

In [ ]:
ds = ds.shuffle()

train_ds = ds["train"].select(range(100000))
validation_ds = ds["train"].skip(80000).select(range(10000))

train_ds = train_ds.map(create_conversation, remove_columns=features)
validation_ds = validation_ds.map(create_conversation, remove_columns=features)
test_ds = ds["test"].map(create_conversation, remove_columns=features)

In [ ]:
train_ds[0]

## SFT

Parameters and their meanings:

1. Training Duration Params:
   1. num_train_epochs: controls the total training duration
   2. max_steps: alternative to epochs, sets max number of training steps
2. Batch size params:
   1. per_device_train_batch_size: determines memory usage and training stability
   2. gradient_accumulation_steps: enables larger effctive batch sizes
3. Learning Rate params:
   1. learning_rate: controlla la dimensione degli update dei pesi
   2. warmup_ratio:
4. Monitoring Params:
   1. logging_steps:
   2. eval_steps
   3. save_steps

In [ ]:
from datasets import load_dataset
from transformers import BitsAndBytesConfig
from trl import SFTConfig, SFTTrainer
import torch
import gc

del model
torch.cuda.empty_cache()
gc.collect()

device = torch.device("cuda")

if torch.cuda.get_device_capability()[0] >= 8:
    torch_dtype = torch.bfloat16
else:
    torch_dtype = torch.float16

model_kwargs = {}


In [ ]:
model_kwargs = dict(
    attn_implementation="eager", # Use "flash_attention_2" when running on Ampere or newer GPU
      torch_dtype=torch_dtype,
)

# BitsAndBytesConfig: Enables 4-bit quantization to reduce model size/memory usage
model_kwargs["quantization_config"] = BitsAndBytesConfig(
    load_in_8bit=True,
)

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained("google/gemma-3-270m", **model_kwargs)

In [ ]:
import wandb
wandb.login()

In [ ]:
%env WANDB_PROJECT=Gemma-Text2SQL

In [ ]:
from peft import LoraConfig

peft_config = LoraConfig(
    lora_alpha=8,
    lora_dropout=0.05,
    r=8,
    bias="none",
    target_modules="all-linear",
    task_type="CAUSAL_LM",
    modules_to_save=["lm_head", "embed_tokens"]
)

In [ ]:
training_args = SFTConfig(
    output_dir="./model/gemma-270m-Text2SQL-Fine-tuned",
    #max_seq_length=512,                     # max sequence length for model and packing of the dataset
    packing=True,                           # Groups multiple samples in the dataset into a single sequence
    num_train_epochs=3,                     # number of training epochs
    per_device_train_batch_size=1,          # batch size per device during training
    gradient_accumulation_steps=4,          # number of steps before performing a backward/update pass
    gradient_checkpointing=True,            # use gradient checkpointing to save memory
    optim="adamw_torch_fused",              # use fused adamw optimizer
    logging_steps=10,                       # log every 10 steps
    save_strategy="epoch",                  # save checkpoint every epoch
    learning_rate=2e-4,                     # learning rate, based on QLoRA paper
    fp16=True if torch_dtype == torch.float16 else False,   # use float16 precision
    bf16=True if torch_dtype == torch.bfloat16 else False,   # use bfloat16 precision
    max_grad_norm=0.3,                      # max gradient norm based on QLoRA paper
    warmup_ratio=0.03,                      # warmup ratio based on QLoRA paper
    lr_scheduler_type="constant",           # use constant learning rate scheduler
    push_to_hub=True,                       # push model to hub
    report_to="wandb",
    run_name="Gemma-Text2SQL"
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=validation_ds,
    processing_class=tokenizer,
    peft_config=peft_config
)

In [ ]:
trainer.train()

In [ ]:
trainer.save_model()

In [ ]:
trainer.push_to_hub("First SFT Completed")

In [ ]:
del model
del trainer
torch.cuda.empty_cache()

In [ ]:
from peft import PeftModel

# Load Model base model
model = AutoModelForCausalLM.from_pretrained("google/gemma-3-270m", low_cpu_mem_usage=True)

# Merge LoRA and base model and save
peft_model = PeftModel.from_pretrained(model, "./model/gemma-270m-Text2SQL-Fine-tuned")
merged_model = peft_model.merge_and_unload()
merged_model.save_pretrained("merged_model", safe_serialization=True, max_shard_size="2GB")

processor = AutoTokenizer.from_pretrained("./model/gemma-270m-Text2SQL-Fine-tuned")
processor.save_pretrained("merged_model")

In [ ]:
merged_model.push_to_hub("leotod/gemma-270m-Text2SQL-Fine-tuned")

In [ ]:
# todo test